# 7. 마을회관과 주요 변수의 최단 거리
- 버스정류장 최단거리
- 병원 최단거리
- 상점 최단거리
- 복지관, 도서관, 영화관 최단거리

In [ ]:
import pandas as pd
from pandas import DataFrame
import numbers
import math
from openpyxl import load_workbook
import numpy as np

In [ ]:
senior_df=pd.read_csv('final_senior_center.csv', encoding='utf-8') # 경로당 데이터
#print(senior_df.head(5))

jeonju_df=senior_df["borough"].str.contains("전주시") # 전주시만 추출

df=senior_df[jeonju_df]
#print(df["address"].tail(2))

In [ ]:
bus_df=pd.read_csv('busStop.csv', encoding='utf-8') # 버스정류장 데이터
#print(bus_df.head(2))

hospital_df=pd.read_csv('Hospital.csv', encoding='utf-8') # 병원 데이터
#print(hospital_df.head(2))

store_df=pd.read_csv('store.csv', encoding='utf-8') # 상점 데이터 마트+시장
#print(store_df.head(2))

welfare_df=pd.read_csv('welfare_center.csv', encoding='utf-8') # 복지관 데이터
#print(welfare_df.head(2))

library_df=pd.read_csv('library.csv', encoding='utf-8') # 도서관 데이터
#print(library_df.head(2))

theater_df=pd.read_csv('theater.csv', encoding='utf-8') # 영화관 데이터
#print(theater_df.head(2))



In [ ]:
class GeoUtil: # x좌표 y좌표 km로 변환해서 x1 y1 x2 y2 차이 구하기
    """
    Geographical Utils
    """
    @staticmethod
    def degree2radius(degree):
        return degree * (math.pi / 180)

    @staticmethod
    def get_harversion_distance(x1, y1, x2, y2, round_decimal_digits=5):
        """
        경위도 (x1,y1)과 (x2,y2) 점의 거리를 반환
        Harversion Formula 이용하여 2개의 경위도간 거래를 구함(단위:Km)
        """
        if x1 is None or y1 is None or x2 is None or y2 is None:
            return None

        R = 6371  # 지구의 반경(단위: km)
        dLon = GeoUtil.degree2radius(x2 - x1)
        dLat = GeoUtil.degree2radius(y2 - y1)

        a = math.sin(dLat / 2) * math.sin(dLat / 2) \
            + (math.cos(GeoUtil.degree2radius(y1)) \
               * math.cos(GeoUtil.degree2radius(y2)) \
               * math.sin(dLon / 2) * math.sin(dLon / 2))
        b = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
        return round(R * b, round_decimal_digits)

In [ ]:
center_x=df['X'] # 경로당(마을회관) x 좌표
center_y=df['Y'] # 경로당(마을회관) y 좌표

busStop_x=bus_df['X'] # 버스정류장 x 좌표
busStop_y=bus_df['Y'] # 버스정류장 y 좌표

hospital_x=hospital_df['X'] # 병원 x 좌표
hospital_y=hospital_df['Y'] # 병원 y 좌표

store_x=store_df['X'] # 상점 x 좌표
store_y=store_df['Y'] # 상점 y 좌표

welfare_x=welfare_df['Longitude'] # 복지관 x 좌표
welfare_y=welfare_df['Latitude'] # 복지관 y 좌표

library_x=library_df['X'] # 도서관 x 좌표
library_y=library_df['Y'] # 도서관 y 좌표

theater_x=theater_df['X'] # 영화관 x 좌표
theater_y=theater_df['Y'] # 영화관 y 좌표

In [ ]:
# 최단 거리 구하기 함수
def distance_func(center_df, parameter_df, X, Y):
    min_distance=10000 # 최솟 거리 초기화
    distance=[] # 최단거리 리스트

    for k in range(len(center_df)): # 리스트 초기화
        distance.append(0)
        
    # 경로당에서 제일 가까운 변수의 거리
    for i in range(0, len(center_df)):
        for j in range(0, len(parameter_df)):
            if (GeoUtil.get_harversion_distance(center_x[i], center_y[i], X[j], Y[j])) <= min_distance:
                min_distance = GeoUtil.get_harversion_distance(center_x[i], center_y[i], X[j], Y[j])
        distance[i]=min_distance
        min_distance=10000
        
    
    distance_df=pd.DataFrame(distance) # 리스트 -> 데이터프레임
    
    return distance_df # 최단 거리 dataframe 반환

### 마을회관과 '버스정류장' 최단거리

In [ ]:
busStop=distance_func(df, bus_df, busStop_x, busStop_y) # 버스정류장

busStop.columns=['busStop_distance'] # column name 변환

#print(busStop.head(10))
#print(len(busStop))

print("버스정류장 평균 거리: ", busStop.mean(), "\n버스정류장 최대 거리: ", busStop.max(), "\n버스정류장 최소 거리: ", busStop.min())


### 마을회관과 '병원' 최단거리

In [ ]:
hospital=distance_func(df, hospital_df, hospital_x, hospital_y) # 병원 

hospital.columns=['hospital_distance'] # column name 변환

print(hospital.head(10))
print(len(hospital))

print("병원 평균 거리: ", hospital.mean(), "\n병원 최대 거리: ", hospital.max(), "\n병원 최소 거리: ", hospital.min())


### 마을회관과 '상권' 최단거리

In [ ]:
store=distance_func(df, store_df, store_x, store_y) # 상점

store.columns=['store_distance'] # column name 변환

print(store.head(10))
print(len(store))

print("상점 평균 거리: ", store.mean(), "\n상점 최대 거리: ", store.max(), "\n상점 최소 거리: ", store.min())


### 마을회관과 '복지관' 최단거리

In [ ]:
welfare=distance_func(df, welfare_df, welfare_x, welfare_y) # 복지관

welfare.columns=['welfare_distance'] # column name 변환

print(welfare.head(10))
print(len(welfare))

print("복지관 평균 거리: ", welfare.mean(), "\n복지관 최대 거리: ", welfare.max(), "\n복지관 최소 거리: ", welfare.min())


### 마을회관과 '도서관' 최단거리

In [ ]:
library=distance_func(df, library_df, library_x, library_y) # 도서관

library.columns=['library_distance'] # column name 변환

print(library.head(10))
print(len(library))

print("도서관 평균 거리: ", library.mean(), "\n도서관 최대 거리: ", library.max(), "\n도서관 최소 거리: ", library.min())


### 마을회관과 '영화관' 최단거리

In [ ]:
theater=distance_func(df, theater_df, theater_x, theater_y) # 영화관

theater.columns=['theater_distance'] # column name 변환

print(theater.head(10))
print(len(theater))

print("영화관 평균 거리: ", theater.mean(), "\n영화관 최대 거리: ", theater.max(), "\n영화관 최소 거리: ", theater.min())


In [ ]:
# 전처리한 데이터 합치기
final_df=pd.concat([df, busStop, hospital, store, welfare, library, theater],axis=1, join='inner')
alldf=final_df.dropna(axis=1)
print(alldf.head(5))

In [ ]:
alldf.to_csv("final_distance.csv", encoding='euc-kr') # csv 변환

### '동' 기준으로 변환

In [ ]:
# 동별 dictionary
def convert_To_dong_dict(parameter_df, parameter_distance):
    
    # parameter_distance dong별 분류
    
    # 완산구 행정동 거리 평균값
    중앙동_mean=(parameter_df[parameter_distance].groupby(df["dong"]=="중앙동")).mean()
    풍남동_mean=(parameter_df[parameter_distance].groupby(df["dong"]=="풍남동")).mean()
    노송동_mean=(parameter_df[parameter_distance].groupby(df["dong"]=="노송동")).mean()
    완산동_mean=(parameter_df[parameter_distance].groupby(df["dong"]=="완산동")).mean()
    동서학동_mean=(parameter_df[parameter_distance].groupby(df["dong"]=="동서학동")).mean()
    서서학동_mean=(parameter_df[parameter_distance].groupby(df["dong"]=="서서학동")).mean()
    중화산동_mean=(parameter_df[parameter_distance].groupby(df["dong"]=="중화산동")).mean()
    평화동_mean=(parameter_df[parameter_distance].groupby(df["dong"]=="평화동")).mean()
    서신동_mean=(parameter_df[parameter_distance].groupby(df["dong"]=="서신동")).mean()
    삼천동_mean=(parameter_df[parameter_distance].groupby(df["dong"]=="삼천동")).mean()
    효자동_mean=(parameter_df[parameter_distance].groupby(df["dong"]=="효자동")).mean()

    # 덕진구 행정도 거리 평균값
    진북동_mean=(parameter_df[parameter_distance].groupby(df["dong"]=="진북동")).mean()
    인후동_mean=(parameter_df[parameter_distance].groupby(df["dong"]=="인후동")).mean()
    덕진동_mean=(parameter_df[parameter_distance].groupby(df["dong"]=="덕진동")).mean()
    금암동_mean=(parameter_df[parameter_distance].groupby(df["dong"]=="금암동")).mean()
    팔복동_mean=(parameter_df[parameter_distance].groupby(df["dong"]=="팔복동")).mean()
    우아동_mean=(parameter_df[parameter_distance].groupby(df["dong"]=="우아동")).mean()
    호성동_mean=(parameter_df[parameter_distance].groupby(df["dong"]=="호성동")).mean()
    송천동_mean=(parameter_df[parameter_distance].groupby(df["dong"]=="송천동")).mean()
    조촌동_mean=(parameter_df[parameter_distance].groupby(df["dong"]=="조촌동")).mean()
    여의동_mean=(parameter_df[parameter_distance].groupby(df["dong"]=="여의동")).mean()
    혁신동_mean=(parameter_df[parameter_distance].groupby(df["dong"]=="혁신동")).mean()
    
    # dictionary 만들기
    
    wansan_dict={"중앙동": 중앙동_mean, "풍남동": 풍남동_mean, "노송동": 노송동_mean, "완산동": 완산동_mean, "동서학동": 동서학동_mean, "서서학동": 서서학동_mean,
             "중화산동": 중화산동_mean, "평화동": 평화동_mean, "서신동": 서신동_mean, "삼천동": 삼천동_mean, "효자동": 효자동_mean,}

    deokjin_dict={"진북동": 진북동_mean, "인후동": 인후동_mean, "덕진동": 덕진동_mean, "금암동": 금암동_mean, "팔복동": 팔복동_mean, 
              "우아동": 우아동_mean, "호성동": 호성동_mean, "송천동": 송천동_mean, "조촌동": 조촌동_mean, "여의동": 여의동_mean, "혁신동": 혁신동_mean}
    
    # dictionary -> dataframe 변환
    wansan=pd.DataFrame(wansan_dict)
    deokjin=pd.DataFrame(deokjin_dict)
    
    # 행 열 transpose
    wansan=np.transpose(wansan)
    deokjin=np.transpose(deokjin)
    
    dong_distance=pd.concat([wansan,deokjin],axis=0, join='inner') # 완산구 + 덕진구 거리 데이터프레임
    dong_distance=dong_distance.dropna(axis=1)
    #print(dong_distance.head(5))

    dong_distance=dong_distance.drop([False], axis='columns') # False열 삭제
    
    return dong_distance

In [ ]:
# 버스정류장 최단거리
busStop_df=convert_To_dong_dict(busStop, "busStop_distance")

busStop_df.rename(columns={True:"busStop_distance"}, inplace=True)
print(busStop_df.head(2))

In [ ]:
# 병원 최단거리
hospital_df=convert_To_dong_dict(hospital, "hospital_distance")

hospital_df.rename(columns={True:"hospital_distance"}, inplace=True)
print(hospital_df.head(2))


In [ ]:
# 상점 최단거리
store_df=convert_To_dong_dict(store, "store_distance")

store_df.rename(columns={True:"store_distance"}, inplace=True)
print(store_df.head(2))

In [ ]:
# 복지관 최단거리
welfare_df=convert_To_dong_dict(welfare, "welfare_distance")

welfare_df.rename(columns={True:"welfare_distance"}, inplace=True)
print(welfare_df.head(2))

In [ ]:
# 도서관
library_df=convert_To_dong_dict(library, "library_distance")

library_df.rename(columns={True:"library_distance"}, inplace=True)
print(library_df.head(2))

In [ ]:
# 영화관
theater_df=convert_To_dong_dict(theater, "theater_distance")

theater_df.rename(columns={True:"theater_distance"}, inplace=True)
print(theater_df.head(2))

In [ ]:
# '동' 기준 dataframe -> 최종 합산

final_distance_dong=pd.concat([busStop_df, hospital_df, store_df, welfare_df, library_df, theater_df],axis=1, join='inner')
final_distance_dong

In [ ]:
final_distance_dong.to_csv("final_distance_dong.csv", encoding='euc-kr') # csv 변환